In [1]:
import joblib
import re
import ipywidgets as widgets
from IPython.display import display
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# 1. SETUP: Download necessary NLTK data if not present
try:
    stop_words = set(stopwords.words('english'))
except:
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))

stemmer = PorterStemmer()

# 2. UPDATED PREPROCESSING: This matches my re-trained model logic
def preprocess_text_final(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters and lowercase
    text = re.sub(r'[^a-zA-Z]', ' ', text).lower()

    words = text.split()

    # Keep negation words so "not good" stays "not good"
    standard_stopwords = set(stopwords.words('english'))
    negation_words = {'not', 'no', 'never', 'neither', 'nor', 'none'}
    custom_stopwords = standard_stopwords - negation_words

    # Stemming and cleaning
    cleaned_words = [stemmer.stem(w) for w in words if w not in custom_stopwords]
    return " ".join(cleaned_words)

# 3. LOAD: Loading my optimized .joblib files
try:
    model = joblib.load('svm_sentiment_model.joblib')
    tfidf = joblib.load('tfidf_vectorizer.joblib')
    print("✅ Model & Vectorizer loaded successfully!")
except FileNotFoundError:
    print("❌ Error: .joblib files not found in the current folder.")

# 4. INTERFACE: Create the UI elements
text_area = widgets.Textarea(
    value='',
    placeholder='Enter your movie review here (e.g., "The plot was not good")...',
    description='Review:',
    layout={'height': '120px', 'width': '90%'}
)
predict_btn = widgets.Button(description="Analyze Sentiment", button_style='primary')
output_display = widgets.Output()

# 5. PREDICTION LOGIC
def on_click_predict(b):
    with output_display:
        output_display.clear_output()
        user_input = text_area.value

        if not user_input.strip():
            print("Please enter a review to analyze.")
            return

        # Process the input
        clean_input = preprocess_text_final(user_input)
        vectorized_input = tfidf.transform([clean_input])

        # Get prediction
        prediction = model.predict(vectorized_input)[0]

        # Format the result
        label = "POSITIVE" if prediction == 1 else "NEGATIVE"
        color = "#28a745" if label == "POSITIVE" else "#dc3545" # Green vs Red

        display(widgets.HTML(f"""
            <div style="padding: 15px; border-radius: 5px; background-color: #f8f9fa; border-left: 5px solid {color};">
                <p style="margin: 0; font-size: 16px;">The model classifies this as:</p>
                <h2 style="color: {color}; margin: 5px 0;">{label}</h2>
            </div>
        """))

# 6. DISPLAY
predict_btn.on_click(on_click_predict)
display(text_area, predict_btn, output_display)

✅ Model & Vectorizer loaded successfully!


Textarea(value='', description='Review:', layout=Layout(height='120px', width='90%'), placeholder='Enter your …

Button(button_style='primary', description='Analyze Sentiment', style=ButtonStyle())

Output()